In [1]:
from fin_statement_model.core.financial_statement import FinancialStatementGraph

In [2]:
historical_periods = ["FY2020", "FY2021", "FY2022"]
forecast_periods = ["FY2023", "FY2024","FY2025"]

revenue_data = {"FY2020": 1000.0, "FY2021": 1100.0, "FY2022": 1210.0}
cogs_data = {p: revenue_data[p] * 0.4 for p in historical_periods}
opex_data = {p: revenue_data[p] * 0.2 for p in historical_periods}
interest_data = {p: 50.0 for p in historical_periods}
tax_data = {p: 0.1 for p in historical_periods}
net_income_data = {p: (revenue_data[p] - cogs_data[p] - opex_data[p] - interest_data[p] - tax_data[p])
                    for p in historical_periods}

In [3]:
#Intialize the graph
fsg = FinancialStatementGraph(periods=[*historical_periods, *forecast_periods])

##Add historical data
# Add financial statement items for each regional revenue
fsg.add_financial_statement_item("revenue_americas", revenue_data)
fsg.add_financial_statement_item("revenue_europe", revenue_data)
fsg.add_financial_statement_item("revenue_apac", revenue_data)

# Now define a calculation node for total revenue as the sum of these three
fsg.add_calculation("revenue", ["revenue_americas", "revenue_europe", "revenue_apac"], "addition")

fsg.add_financial_statement_item("cost_of_goods_sold", cogs_data)
fsg.add_financial_statement_item("operating_expenses", opex_data)
fsg.add_financial_statement_item("non_operating_income", interest_data)
fsg.add_financial_statement_item("non_operating_expenses", interest_data)

fsg.add_financial_statement_item("total_interest_expense", interest_data)
fsg.add_financial_statement_item("tax_expense", tax_data)

# For demonstration, assume these needed raw nodes exist or add dummy data
fsg.add_financial_statement_item("shares_outstanding", {"FY2020": 1000, "FY2021": 1000, "FY2022": 1000})
fsg.add_financial_statement_item("total_assets", {"FY2020": 5000, "FY2021": 5000, "FY2022": 5500})
fsg.add_financial_statement_item("total_equity", {"FY2020": 3000, "FY2021": 3000, "FY2022": 3200})

Adapter type 'excel' is already registered. Overwriting.
Adapter type 'fmp' is already registered. Overwriting.


In [5]:
curve_rates = [0.05, 0.06, 0.07]
fsg.create_forecast(forecast_periods, {"revenue_apac": curve_rates}, method="curve")


# Recalculate all values in the graph
fsg.recalculate_all()

Error creating forecast: Curve method requires rate_progression for node revenue_americas


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/joneickmeier/projects/fin_statement_model/fin_statement_model/core/financial_statement.py", line 202, in create_forecast
  File "/Users/joneickmeier/projects/fin_statement_model/fin_statement_model/core/financial_statement.py", line 231, in _forecast_node
    elif node_method == 'curve':
ValueError: Curve method requires rate_progression for node revenue_americas

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/joneickmeier/projects/fin_statement_model/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/20/nm1z0qb90hvcnsbr53t87xt40000gn/T/ipykernel_8046/317950045.py", line 2, in <module>
    fsg.create_forecast(forecast_periods, {"revenue_apac": curve_rates}, method="curve")
  File "/Users/joneickmeier/projects/fin_statement_model/fin_statement_model/cor

In [5]:
fsg.add_metric( "operating_income")
fsg.add_metric( "gross_profit")
fsg.add_metric( "ebit")
fsg.add_metric( "net_income")
fsg.add_metric( "gross_profit_margin")

ValueError: Required input 'gross_profit' for metric 'operating_income' not found in graph.

In [6]:
print(fsg)

In [7]:
df = fsg.to_dataframe()
df

,FY2020,FY2021,FY2022
revenue_americas,1000.0,1100.0,1210.0
revenue_europe,1000.0,1100.0,1210.0
revenue_apac,1000.0,1100.0,1210.0
revenue,3000.0,3300.0,3630.0
cost_of_goods_sold,400.0,440.0,484.0
operating_expenses,200.0,220.0,242.0
non_operating_income,50.0,50.0,50.0
non_operating_expenses,50.0,50.0,50.0
total_interest_expense,50.0,50.0,50.0
tax_expense,0.1,0.1,0.1


In [8]:
df.pct_change(axis=1)

,FY2020,FY2021,FY2022
revenue_americas,NaN,0.1,0.100000
revenue_europe,NaN,0.1,0.100000
revenue_apac,NaN,0.1,0.100000
revenue,NaN,0.1,0.100000
cost_of_goods_sold,NaN,0.1,0.100000
operating_expenses,NaN,0.1,0.100000
non_operating_income,NaN,0.0,0.000000
non_operating_expenses,NaN,0.0,0.000000
total_interest_expense,NaN,0.0,0.000000
tax_expense,NaN,0.0,0.000000


In [9]:
df.loc["gross_profit_margin",:].plot()


KeyError: 'gross_profit_margin'